In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 150)

In [ ]:
# ---- Task 1.1: Load & Inspect ----
url = "https://raw.githubusercontent.com/PacktPublishing/Hands-on-Exploratory-Data-Analysis-with-Python/refs/heads/master/Chapter%204/sales.csv"

try:
    df_sales = pd.read_csv(url)
except UnicodeDecodeError:
    df_sales = pd.read_csv(url, encoding='ISO-8859-1')

print("Shape:", df_sales.shape)
display(df_sales.head())

print("\nDataFrame info:")
df_sales.info()


Shape: (10000, 9)


,Account,Company,Order,SKU,Country,Year,Quantity,UnitPrice,transactionComplete
0,123456779,Kulas Inc,99985,s9-supercomputer,Aruba,1981,5148,545,False
1,123456784,GitHub,99986,s4-supercomputer,Brazil,2001,3262,383,False
2,123456782,Kulas Inc,99990,s10-supercomputer,Montserrat,1973,9119,407,True
3,123456783,My SQ Man,99999,s1-supercomputer,El Salvador,2015,3097,615,False
4,123456787,ABC Dogma,99996,s6-supercomputer,Poland,1970,3356,91,True



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Account              10000 non-null  int64 
 1   Company              10000 non-null  object
 2   Order                10000 non-null  int64 
 3   SKU                  10000 non-null  object
 4   Country              10000 non-null  object
 5   Year                 10000 non-null  int64 
 6   Quantity             10000 non-null  int64 
 7   UnitPrice            10000 non-null  int64 
 8   transactionComplete  10000 non-null  bool  
dtypes: bool(1), int64(5), object(3)
memory usage: 634.9+ KB


In [ ]:
# ---- Task 1.2: Create TotalPrice ----
# Ensure numeric types (coerce invalids to NaN to avoid string multiplication)
for col in ["Quantity", "UnitPrice"]:
    df_sales[col] = pd.to_numeric(df_sales[col], errors="coerce")

df_sales["TotalPrice"] = df_sales["Quantity"] * df_sales["UnitPrice"]

display(df_sales[["Quantity", "UnitPrice", "TotalPrice"]].head())

,Quantity,UnitPrice,TotalPrice
0,5148,545,2805660
1,3262,383,1249346
2,9119,407,3711433
3,3097,615,1904655
4,3356,91,305396


In [ ]:
# ---- Task 1.3: Outliers ----
outlier_threshold = 3_000_000

n_outliers = (df_sales["TotalPrice"] > outlier_threshold).sum(skipna=True)

df_cleaned_outliers = df_sales.loc[~(df_sales["TotalPrice"] > outlier_threshold)].copy()
row_diff = len(df_sales) - len(df_cleaned_outliers)

print(f"Outlier threshold: {outlier_threshold:,}")
print(f"Number of outlier transactions: {n_outliers}")
print(f"Original rows: {len(df_sales):,}")
print(f"Filtered rows: {len(df_cleaned_outliers):,}")
print(f"Rows removed (outliers): {row_diff:,}")

Outlier threshold: 3,000,000
Number of outlier transactions: 2094
Original rows: 10,000
Filtered rows: 7,906
Rows removed (outliers): 2,094


In [ ]:
# ---- Task A.1: Deduplication ----
dup_count = df_sales.duplicated().sum()
df_no_duplicates = df_sales.drop_duplicates(keep="last").copy()

print(f"Duplicated rows found: {dup_count:,}")
print(f"Original shape: {df_sales.shape}")
print(f"Shape after removing duplicates (keep='last'): {df_no_duplicates.shape}")


Duplicated rows found: 0
Original shape: (10000, 10)
Shape after removing duplicates (keep='last'): (10000, 10)


In [ ]:
# ---- Task A.2: Missing Data ----
missing_per_col = df_no_duplicates.isna().sum().sort_values(ascending=False)
print("Missing values per column:")
display(missing_per_col.to_frame("missing_count").T if len(missing_per_col) < 15 else missing_per_col)

tmp_dropna = df_no_duplicates.dropna()
rows_lost = len(df_no_duplicates) - len(tmp_dropna)
print(f"Rows lost after dropna(): {rows_lost:,}")

if "CustomerID" in df_no_duplicates.columns:
    # Convert to string dtype to safely hold 'ANON'
    df_no_duplicates["CustomerID"] = df_no_duplicates["CustomerID"].astype("string")
    df_no_duplicates["CustomerID"] = df_no_duplicates["CustomerID"].fillna("ANON")
    print("\nSample of CustomerID after fillna('ANON'):")
    display(df_no_duplicates["CustomerID"].head(10))
else:
    print("Column 'CustomerID' not found in the dataset.")

Missing values per column:


,Account,Company,Order,SKU,Country,Year,Quantity,UnitPrice,transactionComplete,TotalPrice
missing_count,0,0,0,0,0,0,0,0,0,0


Rows lost after dropna(): 0
Column 'CustomerID' not found in the dataset.


In [ ]:
# ---- Task A.3: Binning Quantity ----
labels = ["Low Quantity", "Medium Quantity", "High Quantity"]

qty = pd.to_numeric(df_no_duplicates["Quantity"], errors="coerce")

df_no_duplicates["Quantity_Bin"] = pd.cut(qty, bins=3, labels=labels, include_lowest=True)

print("Quantity_Bin value counts:")
display(df_no_duplicates["Quantity_Bin"].value_counts(dropna=False).rename_axis("Quantity_Bin").to_frame("count"))

Quantity_Bin value counts:


,count
Quantity_Bin,
Low Quantity,3357
Medium Quantity,3339
High Quantity,3304



### Task B.1: Classification by Statistical Type

| Variable Name | Statistical Type | Sub-Type |
|---|---|---|
| `UnitPrice` | Quantitative | Continuous |
| `Quantity` | Quantitative | Discrete |
| `Description` | Categorical | Nominal |
| `Country` | Categorical | Nominal |
| `TotalPrice` | Quantitative | Continuous |
| `Quantity_Bin` | Categorical | **Ordinal** |

> **Why?** Prices are on a continuous scale; quantities are counts (discrete). `Description` and `Country` are labels with no inherent order (nominal). `Quantity_Bin` is an ordered category (low < medium < high).



### Task B.2: Classification by Role (for a regression predicting `TotalPrice`)
- **Outcome (Dependent) Variable:** `TotalPrice` (transaction value).
- **Candidate Explanatory (Independent) Variables:**  
  1. `Quantity` — more items typically increase total price.  
  2. `UnitPrice` — higher unit price increases total price for fixed quantity.  
  3. `Country` (via dummies) — captures market/geographic effects (pricing, taxes, demand).  
  
> (Alternatives include engineered features like `Quantity_Bin`, but avoid using both `Quantity` and `Quantity_Bin` together in the same linear model due to collinearity.)


In [ ]:
# ---- Task B.3: Country Dummies ----
if "Country" in df_no_duplicates.columns:
    df_country_dummies = pd.get_dummies(df_no_duplicates["Country"], prefix="Country", drop_first=False, dtype=int)
    print("Country dummies shape:", df_country_dummies.shape)
    display(df_country_dummies.head())
else:
    print("Column 'Country' not found in the dataset.")

Country dummies shape: (10000, 243)


,Country_Afghanistan,Country_Albania,Country_Algeria,Country_American Samoa,Country_Andorra,Country_Angola,Country_Anguilla,Country_Antarctica (the territory South of 60 deg S),Country_Antigua and Barbuda,Country_Argentina,Country_Armenia,Country_Aruba,Country_Australia,Country_Austria,Country_Azerbaijan,Country_Bahamas,Country_Bahrain,Country_Bangladesh,Country_Barbados,Country_Belarus,Country_Belgium,Country_Belize,Country_Benin,Country_Bermuda,Country_Bhutan,Country_Bolivia,Country_Bosnia and Herzegovina,Country_Botswana,Country_Bouvet Island (Bouvetoya),Country_Brazil,Country_British Indian Ocean Territory (Chagos Archipelago),Country_British Virgin Islands,Country_Brunei Darussalam,Country_Bulgaria,Country_Burkina Faso,Country_Burundi,Country_Cambodia,Country_Cameroon,Country_Canada,Country_Cape Verde,Country_Cayman Islands,Country_Central African Republic,Country_Chad,Country_Chile,Country_China,Country_Christmas Island,Country_Cocos (Keeling) Islands,Country_Colombia,Country_Comoros,Country_Congo,...,Country_Serbia,Country_Seychelles,Country_Sierra Leone,Country_Singapore,Country_Slovakia (Slovak Republic),Country_Slovenia,Country_Solomon Islands,Country_Somalia,Country_South Africa,Country_South Georgia and the South Sandwich Islands,Country_Spain,Country_Sri Lanka,Country_Sudan,Country_Suriname,Country_Svalbard & Jan Mayen Islands,Country_Swaziland,Country_Sweden,Country_Switzerland,Country_Syrian Arab Republic,Country_Taiwan,Country_Tajikistan,Country_Tanzania,Country_Thailand,Country_Timor-Leste,Country_Togo,Country_Tokelau,Country_Tonga,Country_Trinidad and Tobago,Country_Tunisia,Country_Turkey,Country_Turkmenistan,Country_Turks and Caicos Islands,Country_Tuvalu,Country_Uganda,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States Minor Outlying Islands,Country_United States Virgin Islands,Country_United States of America,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



## Part C: Conclusion — Reliable Variables
Deduplication and handling missing values **directly impact variable reliability**. Duplicates can inflate counts and bias summary statistics, while missing data—if ignored—can distort relationships or reduce sample size in unpredictable ways. By removing duplicates and either imputing or carefully dropping missing values, we reduce measurement error and improve consistency. This ensures that variables represent the underlying phenomena **reliably**, making downstream analyses and models more trustworthy.

> **الخلاصة:** تنظيف البيانات (إزالة التكرارات ومعالجة القيم المفقودة) يزيد من موثوقية المتغيرات ويجعل التحليل والنمذجة أكثر دقة.
